In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score, roc_curve, accuracy_score, confusion_matrix, f1_score, matthews_corrcoef, precision_score, recall_score
from sksurv.ensemble import RandomSurvivalForest
from sksurv.metrics import concordance_index_censored, cumulative_dynamic_auc

# Load dataset
all_features = pd.read_csv("all_features.csv", sep=",", encoding='GBK')

# Create event and duration arrays
print("Creating event and duration arrays...")
y = pd.DataFrame({
    'event': all_features['livetime'].notna().astype(bool),  # Convert to boolean (True if livetime is not NaN)
    'duration': all_features['livetime'].fillna(1000).astype(float)  # If 'livetime' is NaN, set duration to 1000
})
y = y.to_records(index=False)  # Convert to structured array


# Prepare feature matrix
print("Preparing feature matrix...")
X = all_features[['TG/HDL-C', 'tyg-BMI', 'tyg-i', 'APS-III', 'Age', 'Age score', 'BMI', 'Charlson index', 
                  'Chloride', 'Dementia', 'GAS', 'Glucose', 'Hematocrit', 'Malignant cancer', 
                  'OASIS', 'RBC', 'SAPS-II', 'SIRS', 'SOFA', 'Temperature', 'Urea Nitrogen', 'WBC', 'HDL']]

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

# Initialize and fit the RSF model
rsf_model = RandomSurvivalForest(random_state=1)
rsf_model.fit(X_train, y_train)

# 1.Function to plot cumulative dynamic AUC for each feature
def plot_cumulative_dynamic_auc_for_each_feature(rsf_model, X_test, y_test, features):
    times = np.arange(1, 365, 30)  # Define time points (1 day to 365 days with 30-day intervals)

    plt.figure(figsize=(12, 6))
    
    # Iterate over each feature, keeping all features for model prediction
    for feature in features:
        X_test_single_feature = X_test.copy()
        
        # Mask all features except the current one (set others to zero or mean)
        for col in X_test_single_feature.columns:
            if col != feature:
                X_test_single_feature[col] = X_test_single_feature[col].mean()  # Set non-feature columns to zero, or use the column mean
        
        # Predict using the full model with masked features
        auc_scores, _ = cumulative_dynamic_auc(y_train, y_test, rsf_model.predict_survival_function(X_test_single_feature), times)
        
        # Plot AUC for the specific feature
        plt.plot(times, auc_scores, marker='o', label=feature)
    
    plt.xlabel('Time (days)')
    plt.ylabel('Cumulative Dynamic AUC')
    plt.title('Cumulative Dynamic AUC for Each Feature (RSF Model)')
    plt.ylim(0, 1)
    plt.legend(loc='lower right')
    plt.grid(True)
    plt.show()

# Define the features to plot
features = ['TG/HDL-C', 'tyg-BMI', 'tyg-i', 'BMI', 'HDL']

# Plot cumulative dynamic AUC for each feature in RSF model
plot_cumulative_dynamic_auc_for_each_feature(rsf_model, X_test, y_test, features)


# 2.Base model features and improved features
base_features = ['APS-III', 'Age', 'Age score', 'BMI', 'Charlson index', 'Chloride', 'Dementia', 'GAS', 'Glucose', 'Hematocrit', 'Malignant cancer', 
                  'OASIS', 'RBC', 'SAPS-II', 'SIRS', 'SOFA', 'Temperature', 'Urea Nitrogen', 'WBC']
improved_features = ['TG/HDL-C', 'tyg-BMI', 'tyg-i']

# Prepare training and testing data for base and improved models
X_base_train = X_train[base_features]
X_base_test = X_test[base_features]
X_base_plus_TG_train = X_train[base_features + ['TG/HDL-C']]
X_base_plus_TG_test = X_test[base_features + ['TG/HDL-C']]
X_base_plus_BMI_train = X_train[base_features + ['tyg-BMI']]
X_base_plus_BMI_test = X_test[base_features + ['tyg-BMI']]
X_base_plus_tyg_i_train = X_train[base_features + ['tyg-i']]
X_base_plus_tyg_i_test = X_test[base_features + ['tyg-i']]

# Initialize models
rsf_base = RandomSurvivalForest(random_state=1)
rsf_base_plus_TG = RandomSurvivalForest(random_state=1)
rsf_base_plus_BMI = RandomSurvivalForest(random_state=1)
rsf_base_plus_tyg_i = RandomSurvivalForest(random_state=1)

# Fit the models
models = [rsf_base, rsf_base_plus_TG, rsf_base_plus_BMI, rsf_base_plus_tyg_i]
train_data = [X_base_train, X_base_plus_TG_train, X_base_plus_BMI_train, X_base_plus_tyg_i_train]

for model, X_train in zip(models, train_data):
    model.fit(X_train, y_train)

# Plot ROC curves at specific time points
def plot_roc_curves_at_times(base_model, improved_models, X_test_base, y_test, times):
    """Plot ROC curves for base and improved models at specified time points."""
    plt.figure(figsize=(14, 10))
    
    for i, time in enumerate(times):
        plt.subplot(2, 2, i + 1)  # Create a 2x2 subplot layout
        
        # Base model ROC curve
        surv_probs_base = base_model.predict_survival_function(X_test_base)
        prob_at_time_base = np.array([fn(time) for fn in surv_probs_base])
        risk_scores_base = 1 - prob_at_time_base
        auc_base = roc_auc_score(y_test['event'], risk_scores_base)
        fpr_base, tpr_base, _ = roc_curve(y_test['event'], risk_scores_base)
        plt.plot(fpr_base, tpr_base, label=f'Base Model (AUC = {auc_base:.4f})')
        
        # Improved models ROC curve
        for model, X_test_improved, title in improved_models:
            surv_probs = model.predict_survival_function(X_test_improved)
            prob_at_time = np.array([fn(time) for fn in surv_probs])
            risk_scores = 1 - prob_at_time
            auc = roc_auc_score(y_test['event'], risk_scores)
            fpr, tpr, _ = roc_curve(y_test['event'], risk_scores)
            plt.plot(fpr, tpr, label=f'{title} (AUC = {auc:.4f})')
        
        plt.plot([0, 1], [0, 1], 'k--', label='Random')  # Random baseline
        plt.xlabel('False Positive Rate')
        plt.ylabel('True Positive Rate')
        plt.title(f'ROC Curve at {time} days')
        plt.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()

# Define time points and plot ROC curves
times = [30, 90, 180, 365]
improved_models = [
    (rsf_base_plus_TG, X_base_plus_TG_test, 'Base + TG/HDL-C'),
    (rsf_base_plus_BMI, X_base_plus_BMI_test, 'Base + tyg-BMI'),
    (rsf_base_plus_tyg_i, X_base_plus_tyg_i_test, 'Base + tyg-i')
]
plot_roc_curves_at_times(rsf_base, improved_models, X_base_test, y_test, times)

# Compute and save detailed metrics
def compute_metrics(y_true, y_pred):
    """Compute classification metrics."""
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    accuracy = accuracy_score(y_true, y_pred)
    sensitivity = recall_score(y_true, y_pred)  # Sensitivity (Sn)
    specificity = tn / (tn + fp)  # Specificity (Sp)
    ppv = precision_score(y_true, y_pred)  # Positive Predictive Value (PPV)
    npv = tn / (tn + fn)  # Negative Predictive Value (NPV)
    f1 = f1_score(y_true, y_pred)
    mcc = matthews_corrcoef(y_true, y_pred)
    
    return {
        'Accuracy (%)': accuracy * 100,
        'Sensitivity (Sn) (%)': sensitivity * 100,
        'Specificity (Sp) (%)': specificity * 100,
        'PPV (%)': ppv * 100,
        'NPV (%)': npv * 100,
        'F1 Score': f1,
        'MCC': mcc
    }

def plot_and_save_metrics(base_model, improved_models, X_test_base, y_test, times, output_file):
    """Plot ROC curves and save detailed metrics to a CSV file."""
    metrics_list = []  # Ensure metrics_list is initialized
    
    for time in times:
        # Base model predictions and metrics
        surv_probs_base = base_model.predict_survival_function(X_test_base)
        prob_at_time_base = np.array([fn(time) for fn in surv_probs_base])
        risk_scores_base = 1 - prob_at_time_base
        auc_base = roc_auc_score(y_test['event'], risk_scores_base)
        risk_labels_base = (risk_scores_base >= 0.5).astype(int)
        
        base_metrics = compute_metrics(y_test['event'], risk_labels_base)
        base_metrics.update({'Model': 'Base', 'Time (days)': time, 'AUC': auc_base})
        metrics_list.append(base_metrics)
        
        # Improved models predictions and metrics
        for model, X_test_improved, title in improved_models:
            surv_probs = model.predict_survival_function(X_test_improved)
            prob_at_time = np.array([fn(time) for fn in surv_probs])
            risk_scores = 1 - prob_at_time
            auc = roc_auc_score(y_test['event'], risk_scores)
            risk_labels = (risk_scores >= 0.5).astype(int)
            
            model_metrics = compute_metrics(y_test['event'], risk_labels)
            model_metrics.update({'Model': title, 'Time (days)': time, 'AUC': auc})
            metrics_list.append(model_metrics)

    # Save all metrics to a CSV file
    metrics_df = pd.DataFrame(metrics_list)
    metrics_df.to_csv(output_file, index=False)
    print(f"Metrics saved to {output_file}")

plot_and_save_metrics(rsf_base, improved_models, X_base_test, y_test, times, 'roc_metrics.csv')
